# Introduction

All of the MIMO transmission techniques can be devided into two main groups:
![class](https://raw.githubusercontent.com/kirlf/CSP/master/MIMO/assets/mu-mimo-techs.png)

Moreover, [**spatial multiplexing**](https://www.mathworks.com/help/comm/examples/spatial-multiplexing.html) conxept has the extension: multiplexing between users - macro application of spatial multiplexing.


![sch](https://raw.githubusercontent.com/kirlf/CSP/master/MIMO/assets/mu-mimo-scheme.png)

Hence, the main problem is how to reduce **inter-user interference**.


> **NOTE**:
>
> The modeling is developed by [\[1\]](https://ieeexplore.ieee.org/document/1261332). Please, read proposed paper to obtain more information about considered topic.
>
> You can find additional explanation and illustration in [Prof. Martin Haardt's slides](https://web.stanford.edu/group/sarg/Presentations/FAM2_Martin_Haardt.pdf). We are suggesting them also.

One of the solutions is the **Block diagonalization Zero-Forcing algorithm**. Let us provide short theory explanation with example:

![MU](https://raw.githubusercontent.com/kirlf/communication_stuff/master/MIMO/assets/mu-mimo.jpg)

# Python class implementation

In [1]:
import numpy as np 
import pandas as pd

In [2]:
class ZeroForcingBD:
    def __init__(self, H, Mrs_arr):
        Mr, Mt = np.shape(H)
        self.Mr = Mr
        self.Mt = Mt
        self.H = H
        self.Mrs_arr = Mrs_arr
    
    def routines(self, H, idx, shift):
        
        # used in self.process() - See example above for illustration 
        # inputs: 
        #       H - the whole channel matrix
        #       idx - number of receive antennas of the i-th user
        #       shift - how much receive antennas were considered before
        # outputs:
        #       Uidx, Sigmaidx, Vhidx - SVD decomposition of the H_iP_i 
        #       d - rank of the hat H_i
        #       Hidx - H_i (channel matrix for the i-th user)
        
        Hidx = H[0+shift:idx+shift,:]
        del_idx = [i for i in range(0+shift, idx+shift, 1)]
        H_hat_idx = np.delete(H, del_idx, 0)
        d = np.linalg.matrix_rank(H_hat_idx)
        U, Sigma, Vh = np.linalg.svd(H_hat_idx)
        Vhn = Vh[d:, :]
        Vn = np.matrix(Vhn).H
        Pidx = np.dot(Vn, np.matrix(Vn).H)
        Uidx, Sigmaidx, Vhidx = np.linalg.svd(np.dot(Hidx, Pidx)) 
        return Uidx, Sigmaidx, Vhidx, d, Hidx
    
    def process(self):
        
        # used in self.obtain_matrices()
        # outputs:
        #       F - whole filtering (pre-coding) matrix (array of arrays)
        #       D - whole demodulator (post-processing) matrix (array of arrays)
        #       H - the whole channel matrix (array of arrays)
        
        shift = 0
        H = self.H
        F = []
        D = []
        Hs = []
        for idx in self.Mrs_arr:
            Uidx, Sigmaidx, Vhidx, d, Hidx = self.routines(H, idx, shift)
            Vhidx1 = Vhidx[:d,:]
            Fidx = np.matrix(Vhidx1).H
            F.append(Fidx)
            D.append(Uidx)
            Hs.append(Hidx)
            shift = shift + idx
        return F, D, Hs
    
    def obtain_matrices(self):
        
        # used to obtain pre-coding and post-processing matrices
        # outputs:
        #       FF - whole filtering (pre-coding) matrix 
        #       DD - whole demodulator (post-processing) matrix (array of arrays)
        
        F, D, Hs = self.process()
        FF = np.hstack(F)
        # TODO: calculation of the demodulator matrices
        return FF
        

# Testing

## System parameters

In [3]:
Mrs_arr = [3,4] # 1st user have 2 receive antennas, 2nd user - 3 receive antennas.
Mr = sum(Mrs_arr) # total number of the receive antennas 
Mt = 7 # total number of the transmitt antennas
H = (np.random.randn(Mr,Mt) + 1j*np.random.randn(Mr, Mt))/np.sqrt(2); #Rayleigh flat faded channel matrix (MrxMt)

## Class initialization

In [4]:
BD = ZeroForcingBD(H, Mrs_arr)

## Apply the relevant procedure

In [5]:
F, D, Hs = BD.process()
FF = BD.obtain_matrices()

## Transform to the pandas DataFrame for better representation

In [6]:
df = pd.DataFrame(np.dot(H, FF))
df[abs(df).lt(1e-14)] = 0

In [7]:
pd.DataFrame(np.round(np.real(df),100))

,0,1,2,3,4,5,6
0,-0.519507,-1.199983,0.197358,0.067213,0.000000,0.000000,0.000000
1,0.565909,0.225631,0.111967,-0.169232,0.000000,0.000000,0.000000
2,-0.299454,-0.664594,-0.265603,0.556174,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.478960,-1.691735,0.608402,0.886852
4,0.000000,0.000000,0.000000,0.540617,-0.181793,-0.584790,0.245402
5,0.000000,0.000000,0.000000,-1.223787,-0.601285,0.732952,-0.202793
6,0.000000,0.000000,0.000000,0.867216,2.542366,-0.049963,0.461627


# References

[1] Spencer, Quentin H., A. Lee Swindlehurst, and Martin Haardt. "Zero-forcing methods for downlink spatial multiplexing in multiuser MIMO channels." IEEE transactions on signal processing 52.2 (2004): 461-471.